In [1]:
from fastai import *
from fastai.tabular import *
from fastai.docs import *

In [2]:
untar_data(ADULT_PATH)
ADULT_PATH

PosixPath('../data/adult_sample')

In [5]:
df = pd.read_csv(ADULT_PATH/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


In [6]:
tfms = [FillMissing, Categorify]

In [7]:
FillMissing

fastai.tabular.transform.FillMissing

In [8]:
train_df, valid_df = df[:-2000].copy(),df[-2000:].copy()

In [9]:
dep_var = '>=50k'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

In [10]:
data = tabular_data_from_df(ADULT_PATH, train_df, valid_df, dep_var, tfms=tfms, cat_names=cat_names)
print(data.train_ds.cont_names) 

['capital-loss', 'age', 'education-num', 'hours-per-week', 'fnlwgt', 'capital-gain']


In [11]:
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[ 5 12  3  2  1  5  2 40  1]
 [ 1  5  3  1  1  5  2 40  1]
 [ 5  6  7  8  5  4  1  7  1]
 [ 5 12  5 13  3  5  2 40  1]
 [ 5  9  3  5  6  5  1 40  1]]
[[-0.21676035  0.97948813 -0.42159945 -0.03578891 -0.90272    -0.14592563]
 [-0.21676035  2.6637197  -2.7727149  -1.2476529  -0.5824264  -0.14592563]
 [-0.21676035  1.272398   -2.3808622  -0.03578891  0.54314053 -0.14592563]
 [-0.21676035 -1.0708805  -0.42159945 -0.03578891 -0.7149062  -0.14592563]
 [-0.21676035 -0.19215111  0.3621057   1.1760751  -0.6997213   0.83429855]]
[0 0 0 0 1]


In [12]:
learn = get_tabular_learner(data, layers=[200,100], emb_szs={'native-country': 10}, metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)

Total time: 00:04
epoch  train loss  valid loss  accuracy
0      0.318729    0.334071    0.842000  (00:04)

